In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import matplotlib.pyplot as plt


from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

228827136/228813984 [==============================] - 5s 0us/step


In [ ]:
input_shape = (224,224)
batch_size = 32

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=input_shape,
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=input_shape,
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [3]:
'''
GoogLeNet 코드 작성
functional API, subclass model 중 원하는 방식으로 구현
함수를 사용하면 보다 간편하게 구현 가능
'''

def inception_module(input_layer,filter1=64,reduce1=96,filter2=128,reduce2=16,filter3=32,pool = 32):
  inception_1x1 = layers.Conv2D(filter1, (1,1), padding='same', activation='relu')(input_layer)
  inception_3x3_reduce = layers.Conv2D(reduce1, (1,1), padding='same', activation='relu')(input_layer)
  inception_3x3_pad = layers.ZeroPadding2D(padding=(1, 1))(inception_3x3_reduce)
  inception_3x3 = layers.Conv2D(filter2, (3,3), padding='valid', activation='relu')(inception_3x3_pad)
  inception_5x5_reduce = layers.Conv2D(reduce2, (1,1), padding='same', activation='relu')(input_layer)
  inception_5x5_pad = layers.ZeroPadding2D(padding=(2, 2))(inception_5x5_reduce)
  inception_5x5 = layers.Conv2D(filter3, (5,5), padding='valid', activation='relu')(inception_5x5_pad)
  inception_pool = layers.MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same')(input_layer)
  inception_pool_proj = layers.Conv2D(pool, (1,1), padding='same', activation='relu')(inception_pool)
  inception_output = layers.Concatenate()([inception_1x1,inception_3x3,inception_5x5,inception_pool_proj])
  return inception_output

mP_3x3_2v = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid')

model_input = keras.Input(shape=(224, 224,3), name="f_input")
input_pad = layers.ZeroPadding2D(padding=(3, 3))(model_input)
conv1_7x7_s2 = layers.Conv2D(64, (7,7), strides=(2,2), padding='valid', activation='relu')(input_pad)
conv1_zero_pad = layers.ZeroPadding2D(padding=(1, 1))(conv1_7x7_s2)
pool1_3x3_s2 = mP_3x3_2v(conv1_zero_pad)

conv2_3x3_reduce = layers.Conv2D(64, (1,1), padding='same', activation='relu')(pool1_3x3_s2)
conv2_3x3 = layers.Conv2D(192, (3,3), padding='same', activation='relu')(conv2_3x3_reduce)
conv2_zero_pad = layers.ZeroPadding2D(padding=(1, 1))(conv2_3x3)
pool2_3x3_s2 = mP_3x3_2v(conv2_zero_pad)

module_3a = inception_module(pool2_3x3_s2,64,96,128,16,32,32)
module_3b = inception_module(module_3a,128,128,192,32,96,64)
module_3b_output_zero_pad = layers.ZeroPadding2D(padding=(1, 1))(module_3b)
pool3_3x3_s2 = mP_3x3_2v(module_3b_output_zero_pad)
module_4a = inception_module(pool3_3x3_s2,192,96,208,16,48,64)
# loss1
loss1_ave_pool = layers.AveragePooling2D(pool_size=(5,5), strides=(3,3))(module_4a)
loss1_conv = layers.Conv2D(128, (1,1), padding='same', activation='relu')(loss1_ave_pool)
loss1_flat = layers.Flatten()(loss1_conv)
loss1_fc = layers.Dense(1024, activation='relu')(loss1_flat)
loss1_drop_fc = layers.Dropout(rate=0.7)(loss1_fc)
loss1_classifier = layers.Dense(5)(loss1_drop_fc)
loss1_classifier_act = layers.Activation('softmax')(loss1_classifier)
module_4b = inception_module(module_4a,160,112,224,24,48,64)
module_4c = inception_module(module_4b,128,128,256,24,64,64)
module_4d = inception_module(module_4c,112,144,288,32,64,64)
# loss2
loss2_ave_pool = layers.AveragePooling2D(pool_size=(5,5), strides=(3,3))(module_4d)
loss2_conv = layers.Conv2D(128, (1,1), padding='same', activation='relu')(loss2_ave_pool)
loss2_flat = layers.Flatten()(loss2_conv)
loss2_fc = layers.Dense(1024, activation='relu')(loss2_flat)
loss2_drop_fc = layers.Dropout(rate=0.7)(loss2_fc)
loss2_classifier = layers.Dense(5)(loss2_drop_fc)
loss2_classifier_act = layers.Activation('softmax')(loss2_classifier)

module_4e = inception_module(module_4d,256,160,320,32,128,128)
module_4e_output_zero_pad = layers.ZeroPadding2D(padding=(1, 1))(module_4e)
pool4_3x3_s2 = mP_3x3_2v(module_4e_output_zero_pad)
module_5a = inception_module(pool4_3x3_s2,256,160,320,32,128,128)
module_5b = inception_module(module_5a,384,192,384,48,128,128)

# loss3
avg_pool = layers.AveragePooling2D(pool_size=(7,7), strides=(1,1))(module_5b)
loss3_flat = layers.Flatten()(avg_pool)
drop_out = layers.Dropout(rate=0.4)(loss3_flat)
loss3_classifier = layers.Dense(5)(drop_out)
loss3_classifier_act = layers.Activation('softmax')(loss3_classifier)
functional_model = keras.Model(model_input, [loss1_classifier_act,loss2_classifier_act,loss3_classifier_act], name="functional_model")

In [4]:
functional_model.summary()

Model: "functional_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 f_input (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['f_input[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9472        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                   

In [ ]:
functional_model.compile(optimizer='Adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['acc'])
epochs=10
history = functional_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


92/92 [==============================] - 36s 330ms/step - loss: 6.0923 - activation_1_loss: 2.1532 - activation_2_loss: 2.0139 - activation_3_loss: 1.9252 - activation_1_acc: 0.3011 - activation_2_acc: 0.2725 - activation_3_acc: 0.2629 - val_loss: 4.0214 - val_activation_1_loss: 1.3052 - val_activation_2_loss: 1.3384 - val_activation_3_loss: 1.3777 - val_activation_1_acc: 0.4946 - val_activation_2_acc: 0.4891 - val_activation_3_acc: 0.4728
Epoch 2/10
92/92 [==============================] - 30s 321ms/step - loss: 3.7487 - activation_1_loss: 1.2414 - activation_2_loss: 1.2641 - activation_3_loss: 1.2432 - activation_1_acc: 0.4782 - activation_2_acc: 0.4380 - activation_3_acc: 0.4479 - val_loss: 3.6906 - val_activation_1_loss: 1.2380 - val_activation_2_loss: 1.2423 - val_activation_3_loss: 1.2103 - val_activation_1_acc: 0.5559 - val_activation_2_acc: 0.5082 - val_activation_3_acc: 0.5150
Epoch 3/10
92/92 [==============================] - 30s 320ms/step - loss: 3.3965 - activation_1_loss

In [ ]:
history.history

{'activation_1_acc': [0.30108991265296936,
  0.47820162773132324,
  0.5381471514701843,
  0.5899182558059692,
  0.5974114537239075,
  0.6406675577163696,
  0.6600817441940308,
  0.6900544762611389,
  0.7019754648208618,
  0.7224114537239075],
 'activation_1_loss': [2.15317440032959,
  1.2414016723632812,
  1.1270755529403687,
  1.018894910812378,
  1.0190991163253784,
  0.91715407371521,
  0.8622736930847168,
  0.7984067797660828,
  0.7968277931213379,
  0.7365976572036743],
 'activation_2_acc': [0.2724795639514923,
  0.43801090121269226,
  0.5156675577163696,
  0.5923024415969849,
  0.5923024415969849,
  0.6253405809402466,
  0.6488419771194458,
  0.6839237213134766,
  0.6798365116119385,
  0.7098092436790466],
 'activation_2_loss': [2.013892889022827,
  1.2640935182571411,
  1.1365067958831787,
  1.0222158432006836,
  1.0126255750656128,
  0.917365312576294,
  0.8692910671234131,
  0.8253872394561768,
  0.8244094848632812,
  0.7458422780036926],
 'activation_3_acc': [0.26294279098510